# Financial App

In [1]:
import os
from pathlib import Path
import pandas as pd

# Get the current working directory
root_path = Path(os.getcwd())
if  root_path.name == "notebooks":
    print("✅ La última carpeta es 'notebooks'.")
    os.chdir(f"{root_path}/..")
    print(f"Now in directory: {os.getcwd()}")
else:
    print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
import app.modules.database.structur_databases as sdb
import app.modules.database.companies as companies
import app.modules.database.customers as customers
import app.modules.database.credit_manager as credit_manager
import app.modules.database.portfolio_manager as portfolio_manager
import app.modules.database.collection as colltions

from app.modules.database.credit_manager import credits_balance

from importlib import reload

reload(customers)
reload(companies)
reload(portfolio_manager)
reload(credit_manager)

%run logs/run_log.py

✅ La última carpeta es 'notebooks'.
Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
✅ The system correctly imports all modules.
✅ SQL script executed successfully.
✅ 24 provinces successfully inserted into the database.
✅ The company NeoCrediT S.A. has been added.
✅ New business plan created, its ID is 1 and it was associated with the company NeoCrediT S.A..
✅ The company Onoyen S.R.L. has been added.
✅ New business plan created, its ID is 2 and it was associated with the company Onoyen S.R.L..
✅ The company Asociación Mutual Union Federal has been added.
✅ New business plan created, its ID is 3 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 4 and it was associated with the company Asociación Mutual Union Federal.
✅ New business plan created, its ID is 5 and it was associated with the company Asociación Mutual Union Federal.
✅ Settings successfully updated in the database.
✅ Supplier 2 

In [2]:
pp = pd.read_sql('portfolio_purchases', engine, index_col='ID')
customers = pd.read_sql('customers', engine, index_col='ID')
credits = pd.read_sql('credits', engine, index_col='ID')
collections = pd.read_sql('collection', engine, index_col='ID')
installments = pd.read_sql('installments', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')
bp = pd.read_sql('business_plan', engine, index_col='ID')
companies = pd.read_sql('companies', engine, index_col='ID')

balance = credits_balance().copy()
print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

balance = balance.loc[(balance['ID_Owner'] == 1)].copy()
balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)

balance

Capital: $ 82,862,148.67
Interés: $ 91,225,098.31
IVA: $ 17,180,558.60
Total: $ 191,267,805.51


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
1,1,1,2022-10-20,$0.00,$0.00,$0.00,$0.00,1
2,1,2,2022-11-20,$0.00,$0.00,$0.00,$0.00,1
3,1,3,2022-12-20,$0.00,$0.00,$0.00,$0.00,1
4,1,4,2023-01-20,"$6,309.57","$11,225.94",$0.00,"$17,535.51",1
5,1,5,2023-02-20,"$6,811.56","$10,811.08",$0.00,"$17,622.63",1
...,...,...,...,...,...,...,...,...
5774,407,11,2026-02-28,"$50,296.54","$38,068.32","$7,994.35","$96,359.21",1
5775,407,12,2026-03-28,"$57,280.81","$32,296.20","$6,782.20","$96,359.21",1
5776,407,13,2026-04-28,"$65,234.93","$25,722.55","$5,401.74","$96,359.21",1


In [3]:
date = "25-03-01"

from app.modules.database.collection import charging, collection_w_early_cancel, TypeDataCollection, myIE, alIE 

# ✅ Load collections report
coll = pd.read_excel(f'inputs/Reporte - Cobranzas - {date}.xlsx')
coll = coll.query("Fondeador != 'ONOYEN'")
coll.drop(columns=["GS", "PU"], inplace=True)

# ✅ Load investment report
inv = pd.read_excel(f'inputs/Reporte - Inv. Créditos - {date}.xlsx', index_col="Id. Op.")
inv = inv.query("Fondeador != 'ONOYEN'")

# ✅ Load existing credits from the database
credits = pd.read_sql("SELECT * FROM credits", engine, index_col="ID")

# ✅ Map external IDs
coll["ID_External"] = coll["Crédito"].map(lambda x: inv.loc[x, "Clave Externa"].strip("CH_") if x in inv.index else None)

# ✅ Map internal IDs
coll["ID"] = coll["ID_External"].map(
    lambda x: credits.loc[credits["ID_External"] == x].index[0] 
    if x in credits["ID_External"].values and not credits.loc[credits["ID_External"] == x].empty 
    else None
)

# ✅ Replace NaN values with placeholder (-1), then convert to integer safely
coll[["ID_External", "ID"]] = coll[["ID_External", "ID"]].fillna(-1).astype(int, errors="ignore")
coll.replace({"ID_External": -1, "ID": -1}, None, inplace=True)

# ✅ Standardize "Tipo Cobranza" values
coll["Tipo Cobranza"] = coll["Tipo Cobranza"].replace("ANTICIPO", "COBRANZA")
coll["Tipo Cobranza"] = coll.apply(lambda row: "PENALTY" if row["Línea"] == "PENALTY" else row["Tipo Cobranza"], axis=1)

# ✅ Ensure correct indexing for missing 'PENALTY' IDs
penalty_index = coll.index[coll["ID"].isna() & (coll["Tipo Cobranza"] == "PENALTY")]

# ✅ Assign unique IDs only to the required rows
coll.loc[penalty_index, "ID"] = range(credits.index.max() + 1, credits.index.max() + 1 + len(penalty_index))

# ✅ Organize and group collection data
coll_org_id = coll.dropna(subset=["ID"]).copy()
coll = coll.groupby(["Emisión", "CUIL", "ID", "Tipo Cobranza"])[["CA", "IN", "IV", "TOTAL"]].sum().reset_index()
coll = coll.sort_values(by=["Emisión", "ID"])

# ✅ Process penalties
penalties = coll.loc[coll['Tipo Cobranza'] == 'PENALTY'].sort_values(by=["Emisión", "ID"]).set_index("ID")
new_penalties = credits.iloc[:0].copy()
new_installments = pd.read_sql("SELECT * FROM installments WHERE 1=0", engine)  # Load an empty DataFrame structure

# ✅ Importing function for handling portfolio purchases
from app.modules.database.portfolio_manager import add_portfolio_purchase

# ✅ Registering a new portfolio purchase
add_portfolio_purchase(pd.Timestamp.now(), 3, 0.0, False, False, True, True)

# ✅ Retrieving the last recorded portfolio purchase ID
pp = pd.read_sql('portfolio_purchases', engine, index_col='ID')
last_pp = pp.index.max()

# ✅ Loading customer data from the database
customers = pd.read_sql('customers', engine, index_col='ID')

# ✅ Processing penalties, creating new credit and installment records
for i in penalties.index:
    new_penalties.loc[i] = {
        'ID_External': None,
        'ID_Client': customers.loc[customers['CUIL'] == penalties.loc[i, 'CUIL']].index.values[0],
        'Date_Settlement': penalties.loc[i, 'Emisión'],
        'ID_BP': 1,
        'Cap_Requested': 0.0,
        'Cap_Grant': 0.0,
        'N_Inst': 1,
        'First_Inst_Purch': 1,
        'TEM_W_IVA': 0.0,
        'V_Inst': -penalties.loc[i, 'TOTAL'],
        'First_Inst_Sold': None,
        'D_F_Due': penalties.loc[i, 'Emisión'],
        'ID_Purch': last_pp,
        'ID_Sale': None
    }
    
    new_installments.loc[i] = {
        'ID_Op': i,
        'Nro_Inst': 1,
        'D_Due': penalties.loc[i, 'Emisión'],
        'Capital': 0.0,
        'Interest': -penalties.loc[i, 'IN'],
        'IVA': -penalties.loc[i, 'IV'],
        'Total': -penalties.loc[i, 'TOTAL'],
        'ID_Owner': 1
    }

# ✅ Saving new penalty credits and installments to the database
new_penalties.to_sql('credits', engine, index=False, if_exists='append')
new_installments.to_sql('installments', engine, index=False, if_exists='append')

# ✅ Generating a DataFrame summarizing collections
cobranzas = pd.DataFrame(
    coll.loc[coll['Tipo Cobranza'] != 'COBRANZA X CANCEL ANT']
    .groupby(['Emisión', 'ID'])['TOTAL']
    .sum()
)

✅ Portfolio purchase ID 10 successfully added for supplier 3.


In [ ]:
for date, id in cobranzas.index:
    if 'COBRANZA X CANCEL ANT' not in coll.loc[coll['Emisión'] == date, 'Tipo Cobranza'].values:
        try:
            charging(TypeDataCollection.ID_Op, id, -cobranzas.loc[(date, id), 'TOTAL'], 3, date, True)
        except myIE or alIE:
            raise print(f'ID Op.: {id} - {date} - $ {-cobranzas.loc[(date, id), 'TOTAL']:,.2f}')
    else:
        try:
            collection_w_early_cancel(TypeDataCollection.ID_Op, id, -cobranzas.loc[(date, id), 'TOTAL'], 3, date, True)
        except myIE or alIE:
            raise print(f'ID Op.: {id} - {date} - $ {-cobranzas.loc[(date, id), 'TOTAL']:,.2f}')

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000240D2A54040>
Traceback (most recent call last):
  File "C:\Program Files\Lib\weakref.py", line 370, in remove
    self = selfref()
KeyboardInterrupt: 


In [ ]:
from app.modules.database.collection import delete_collection_by_id
from app.modules.database.companies import add_bussines_plan

collections = pd.read_sql('collection', engine, index_col='ID')
installments = pd.read_sql('installments', engine, index_col='ID')
for i in collections.loc[collections['ID_Inst'].isin(installments.loc[installments['ID_Op'] == 301].index.values)].index:
    delete_collection_by_id(i)

charging(TypeDataCollection.ID_Op, 301, 57859.57, 3, pd.Timestamp("2024-09-11"), True)
collection_w_early_cancel(TypeDataCollection.ID_Op, 301, 458.91, 3, pd.Timestamp("2024-09-11"), True)
charging(TypeDataCollection.ID_Op, 301, 9949.52, 3, pd.Timestamp("2024-10-09"), True)
charging(TypeDataCollection.ID_Op, 301, 47451.14, 3, pd.Timestamp("2024-10-31"), True)
charging(TypeDataCollection.ID_Op, 301, 764.85, 3, pd.Timestamp("2024-10-31"), True)

new_collections = collections.iloc[0:0].copy()
new_collections.loc[0] = {
    'ID_Inst': 4385,
    'D_Emission': pd.Timestamp("2024-10-31"),
    'Type_Collection': 'ANTICIPADA',
    'Capital': 50765.87 - (38351.16 + 8053.74),
    'Interest': 38351.16,
    'IVA': 8053.74,
    'Total': 50765.87
}

new_collections.to_sql('collection', engine, index=False, if_exists='append')
print('Cartera de créditos AMUF migrada.')

balance = credits_balance()
# Leer y transformar datos
inv = pd.read_excel('inputs/Reporte - Inv. Créditos - 24-12-31.xlsx', index_col='Id. Op.')
inv = inv.loc[inv['Fondeador'] != 'ONOYEN']

credits = pd.read_sql('credits', engine, index_col='ID')
credits.loc[~credits['ID_External'].isna(), 'ID_External'] = credits.loc[~credits['ID_External'].isna(), 'ID_External'].astype(np.int64)

# Manejo robusto de Clave Externa
inv['Clave Externa'] = inv['Clave Externa'].str.split("CH_").apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
inv['Clave Externa'] = pd.to_numeric(inv['Clave Externa'], errors='coerce').astype('Int64')

coll_org_id['ID'] = pd.to_numeric(coll_org_id['ID'], errors='coerce').astype('Int64')

# Validar discrepancias
for id in coll_org_id['ID_External'].dropna().unique():
    imp = inv.loc[inv['Clave Externa'] == id, 'Cap. Cuotas Imp. (V/aV)'].sum()
    bal = balance.loc[balance['ID_Op'] == credits.loc[credits['ID_External'] == id].index.values[0], 'Capital'].sum()
    if abs(imp - bal) > 1:
        print(f'Discrepancia en Capital para ID {id}: Calculado $ {imp:,.2f} - Reportado $ {bal:,.2f}')

    imp = inv.loc[inv['Clave Externa'] == id, 'Int. Cuotas Imp. (V/aV)'].sum()
    bal = balance.loc[balance['ID_Op'] == credits.loc[credits['ID_External'] == id].index.values[0], 'Interest'].sum()
    if abs(imp - bal) > 1:
        print(f'Discrepancia en Interés para ID {id}: Calculado $ {imp:,.2f} - Reportado $ {bal:,.2f}')

    imp = inv.loc[inv['Clave Externa'] == id, 'Iva Cuotas Imp. (V/aV)'].sum()
    bal = balance.loc[balance['ID_Op'] == credits.loc[credits['ID_External'] == id].index.values[0], 'IVA'].sum()
    if abs(imp - bal) > 1:
        print(f'Discrepancia en IVA para ID {id}: Calculado $ {imp:,.2f} - Reportado $ {bal:,.2f}')

    imp = inv.loc[inv['Clave Externa'] == id, 'Imp. Cuotas Imp. (V/aV)'].sum()
    bal = balance.loc[balance['ID_Op'] == credits.loc[credits['ID_External'] == id].index.values[0], 'Total'].sum()
    if abs(imp - bal) > 1:
        print(f'Discrepancia en Total para ID {id}: Calculado $ {imp:,.2f} - Reportado $ {bal:,.2f}')

imp = inv['Imp. Cuotas Imp. (V/aV)'].sum()
if abs(imp - balance['Total'].sum()) < 10:
    print('Cartera de AMUF migrada correctamente.')

✅ Collection record with ID 2166 successfully deleted.
✅ Collection record with ID 2167 successfully deleted.
✅ Collection record with ID 2403 successfully deleted.
✅ Collection record with ID 2438 successfully deleted.
✅ Collection record with ID 2439 successfully deleted.
✅ Collection record with ID 2969 successfully deleted.
✅ Collection record with ID 3108 successfully deleted.
✅ Collection record with ID 3109 successfully deleted.
✅ Collection record with ID 3110 successfully deleted.
✅ Collection record with ID 3198 successfully deleted.
✅ Collection record with ID 3199 successfully deleted.
✅ Collection record with ID 3408 successfully deleted.
Cartera de créditos AMUF migrada.
Discrepancia en Capital para ID 80906: Calculado $ 38,066.64 - Reportado $ 0.00
Discrepancia en Interés para ID 80906: Calculado $ 6,296.32 - Reportado $ 0.00
Discrepancia en IVA para ID 80906: Calculado $ 1,322.23 - Reportado $ 0.00
Discrepancia en Total para ID 80906: Calculado $ 45,685.19 - Reportado $

In [ ]:
balance = credits_balance().copy()
print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

balance = balance.loc[(balance['ID_Owner'] == 1)].copy()
balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)

balance

Capital: $ 58,014,183.05
Interés: $ 48,939,877.42
IVA: $ 8,300,662.41
Total: $ 115,254,722.52


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
1,1,1,2022-10-20,$0.00,$0.00,$0.00,$0.00,1
2,1,2,2022-11-20,$0.00,$0.00,$0.00,$0.00,1
3,1,3,2022-12-20,$0.00,$0.00,$0.00,$0.00,1
4,1,4,2023-01-20,"$6,309.57","$11,225.94",$0.00,"$17,535.51",1
5,1,5,2023-02-20,"$6,811.56","$10,811.08",$0.00,"$17,622.63",1
...,...,...,...,...,...,...,...,...
5839,468,1,2025-01-17,$0.00,$0.00,$0.00,$0.00,1
5840,469,1,2025-01-17,$0.00,$0.00,$0.00,$0.00,1
5841,470,1,2025-02-17,$0.00,$0.00,$0.00,$0.00,1
